In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from pandas.core.frame import DataFrame

In [2]:
#用pandas读入数据
#记得改路径
# train_data_ads = pd.read_csv('../dataset/train.csv')
# train_data_feeds = pd.read_csv('../dataset/train_feeds.csv')

# test_data_ads = pd.read_csv('../dataset/final_test.csv')
# test_data_feeds = pd.read_csv('../dataset/test_feeds.csv')

train_data = pd.read_csv('../dataset/train.csv')
train_feed = pd.read_csv('../dataset/train_feeds.csv')
test_data = pd.read_csv('../dataset/final_test.csv')
test_feed = pd.read_csv('../dataset/test_feeds.csv')

In [3]:
# 自然数编码(重新进行编码)---->准确说是统计矩阵型数据种类
def label_encode(series):
    unique = list(series.unique())
    return series.map(dict(zip(unique, range(series.nunique()))))

for col in ['ad_click_list_v001','ad_click_list_v002','ad_click_list_v003','ad_close_list_v001','ad_close_list_v002','ad_close_list_v003','u_newsCatInterestsST']:
    train_data.loc[:,col]= label_encode(train_data.loc[:,col])
    
for col in ['u_newsCatInterests','u_newsCatDislike','u_newsCatInterestsST','u_click_ca2_news']:
    train_feed.loc[:,col]= label_encode(train_feed.loc[:,col])

for col in ['ad_click_list_v001','ad_click_list_v002','ad_click_list_v003','ad_close_list_v001','ad_close_list_v002','ad_close_list_v003','u_newsCatInterestsST']:
    test_data.loc[:,col]= label_encode(test_data.loc[:,col])
    
for col in ['u_newsCatInterests','u_newsCatDislike','u_newsCatInterestsST','u_click_ca2_news']:
    test_feed.loc[:,col]= label_encode(test_feed.loc[:,col])

In [4]:
#数据缺失处理（不具有穿越特征)
train_feed = train_feed.drop(columns=['label','i_docId','i_s_sourceId','i_entities'])
test_feed = test_feed.drop(columns=['i_docId','i_s_sourceId','i_entities'])

train_data.rename(columns={'pt_d': 'e_et','user_id':'u_userId'}, inplace=True)
test_data.rename(columns={'pt_d': 'e_et','user_id':'u_userId'}, inplace=True)

train_data=train_data.dropna(subset='e_et')
train_feed=train_feed.dropna(subset='e_et')
test_data=test_data.dropna(subset='e_et')
test_feed=test_feed.dropna(subset='e_et')

train_feed = train_feed.drop_duplicates(keep='last', subset=['u_userId', 'e_et'])
test_feed = test_feed.drop_duplicates(keep='last', subset=['u_userId', 'e_et'])

train_data['e_et']=train_data['e_et'].astype(int)
train_feed['e_et']=train_feed['e_et'].astype(int)
test_data['e_et']=test_data['e_et'].astype(int)
test_feed['e_et']=test_feed['e_et'].astype(int)

train_data = train_data.sort_values(by='e_et').sort_values(by='u_userId').fillna(method='pad')
train_feed = train_feed.sort_values(by='e_et').sort_values(by='u_userId').fillna(method='pad')
test_data = test_data.sort_values(by='e_et').sort_values(by='u_userId').fillna(method='pad')
test_feed = test_feed.sort_values(by='e_et').sort_values(by='u_userId').fillna(method='pad')

train_data = train_data.merge(train_feed,on=['u_userId','e_et'] ,how='left',sort=False)
test_data = test_data.merge(test_feed,on=['u_userId','e_et'] ,how='left',sort=False)

test_data = test_data.drop(columns=['label'])

C:\Users\76678\AppData\Local\Temp\ipykernel_14928\10440150.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_data = train_data.sort_values(by='e_et').sort_values(by='u_userId').fillna(method='pad')
C:\Users\76678\AppData\Local\Temp\ipykernel_14928\10440150.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_feed = train_feed.sort_values(by='e_et').sort_values(by='u_userId').fillna(method='pad')
C:\Users\76678\AppData\Local\Temp\ipykernel_14928\10440150.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_data = test_data.sort_values(by='e_et').sort_values(by='u_userId').fillna(method='pad')
C:\Users\76678\AppData\Local\Temp\ipykernel_14928\10440150.py:24: FutureWarning: DataFrame.fillna with 'm

In [5]:
# 转换成Dmatrix格式
feature_data_columns = [f for f in train_data.columns if f not in ['log_id','u_userId','e_et','label']]
feature_test_columns = [f for f in test_data.columns if f not in ['log_id','u_userId','e_et','label']]
target_column = 'label'
xgtrain = xgb.DMatrix(train_data[feature_data_columns].values, train_data[target_column].values)
xgtest = xgb.DMatrix(test_data[feature_test_columns].values)

In [7]:
#超参数设定
#警告暂时不用管
#最佳成绩：0.707721
# # gpu版本
# param = {'max_depth':6,
#          'eta':1.1,
#          'verbosity':0,
#          'subsample':1,
#          'colsample_bytree':0.7,
#          'objective':'multi:softprob',
#          'num_class':2,
#          'tree_method':'gpu_hist'}

# cpu版本
param = {'max_depth':10,
         'eta':0.1,
         'verbosity':0,
         'subsample':1,
         'colsample_bytree':0.7,
         'objective':'multi:softprob',
         'num_class':2
         }
#设定watchlist用于查看模型状态
watchlist  = [(xgtrain,'train')]
num_round = 20
bst = xgb.train(param, xgtrain, num_round, watchlist)

[0]	train-mlogloss:0.12701
[1]	train-mlogloss:0.06400
[2]	train-mlogloss:0.04970
[3]	train-mlogloss:0.04691
[4]	train-mlogloss:0.04613
[5]	train-mlogloss:0.04598
[6]	train-mlogloss:0.04565
[7]	train-mlogloss:0.04540
[8]	train-mlogloss:0.04527
[9]	train-mlogloss:0.04520
[10]	train-mlogloss:0.04505


KeyboardInterrupt: 

In [ ]:
#输出
test_pred = pred = bst.predict(xgtest)[:, 1]
df = {'pctr': test_pred}
df = DataFrame(df)
df = df.reset_index()
df.columns = ['id', 'pctr']
df.to_csv('./submission.csv', index=False)

# pred = bst.predict(xgtest)
# test_data['pctr'] = pred[:,1]
# test_data[['log_id', 'pctr']].to_csv('submission.csv',index=None)